### 1. Rank Most Expensive Products per Subcategory
**Functional Specification:**
* Select product name, list price, and product subcategory ID from `Production.Product`.
* Partition the data by `ProductSubcategoryID`.
* Order the partitions by `ListPrice` in descending order.
* Use `RANK()` to assign a rank to each product within its subcategory based on price.
* Filter to show only the top 5 most expensive products per subcategory (requires a CTE or derived table).

In [ ]:
WITH RankedProducts AS (
    SELECT
        p.Name,
        p.ListPrice,
        p.ProductSubcategoryID,
        RANK() OVER (
            PARTITION BY p.ProductSubcategoryID
            ORDER BY p.ListPrice DESC
        ) AS Rank
    FROM Production.Product AS p
)
SELECT
    Name,
    ListPrice,
    ProductSubcategoryID,
    Rank
FROM RankedProducts
WHERE Rank <= 5
ORDER BY ProductSubcategoryID

### 2. Identify Top Salespeople by Territory
**Functional Specification:**
* Join `Sales.SalesPerson` with `Person.Person` to get names.
* Use `ROW_NUMBER()` to create a unique ranking for salespeople.
* Partition by `TerritoryID` to isolate rankings per region.
* Order by `SalesYTD` descending to find the highest performers.
* Return the name, territory, sales YTD, and their rank.

In [ ]:
SELECT
    p.FirstName,
    p.LastName,
    sp.TerritoryID,
    sp.SalesYTD,
    ROW_NUMBER() OVER (
        PARTITION BY sp.TerritoryID
        ORDER BY sp.SalesYTD DESC
    ) AS Rank
FROM Sales.SalesPerson AS sp
JOIN Person.Person AS p
    ON p.BusinessEntityID = sp.BusinessEntityID
ORDER BY sp.TerritoryID

### 3. Segment Customers by Spending Groups (NTILE)
**Functional Specification:**
* Calculate total due from `Sales.SalesOrderHeader` for each `CustomerID`.
* Use `NTILE(4)` to divide customers into 4 buckets (Quartiles) based on their total spending.
* Order the window by the sum of `TotalDue` descending.
* Return CustomerID, TotalSpend, and their SpendingQuartile.

In [ ]:
WITH TotalSpending AS (
    SELECT
        soh.CustomerID,
        SUM(soh.TotalDue) AS TotalSpending
    FROM Sales.SalesOrderHeader AS soh
    GROUP BY soh.CustomerID
)
SELECT
    CustomerID,
    TotalSpending,
    NTILE(4) OVER (ORDER BY TotalSpending DESC) AS SpendingQuartile
FROM TotalSpending

### 4. Latest Order for Each Employee
**Functional Specification:**
* Retrieve `SalesOrderID`, `OrderDate`, and `SalesPersonID`.
* Use `ROW_NUMBER()` to identify the specific latest row.
* Partition by `SalesPersonID`.
* Order by `OrderDate` descending to place the newest order first.
* Select only rows where RowNum = 1 from the derived table.

In [ ]:
SELECT
    s.SalesOrderID,
    s.SalesPersonID,
    s.OrderDate,
    s.RowNum
FROM (
    SELECT
        soh.SalesOrderID,
        soh.SalesPersonID,
        soh.OrderDate,
        ROW_NUMBER() OVER (
            PARTITION BY soh.SalesPersonID
            ORDER BY soh.OrderDate DESC
        ) AS RowNum
    FROM Sales.SalesOrderHeader AS soh
) AS s
WHERE s.RowNum = 1

### 5. Rank Vendors by Credit Rating and Purchasing Activity
**Functional Specification:**
* Join `Purchasing.Vendor` and `Purchasing.PurchaseOrderHeader`.
* Sum the `TotalDue` for all orders per vendor.
* Use `DENSE_RANK()` so that vendors with identical spending totals share a rank without gaps.
* Partition by `CreditRating` to see top vendors within each rating tier.
* Order by total purchase amount descending.

In [ ]:
SELECT
    VendorID,
    CreditRating,
    TotalDue,
    DENSE_RANK() OVER (
        PARTITION BY CreditRating
        ORDER BY TotalDue DESC
    ) AS Rank
FROM (
    SELECT
        v.BusinessEntityID AS VendorID,
        v.CreditRating,
        SUM(poh.TotalDue) AS TotalDue
    FROM Purchasing.Vendor AS v
    JOIN Purchasing.PurchaseOrderHeader AS poh
        ON v.BusinessEntityID = poh.VendorID
    GROUP BY
        v.BusinessEntityID,
        v.CreditRating
) as t
ORDER BY CreditRating

### 6. Department Seniority Ranking
**Functional Specification:**
* Join `HumanResources.Employee`, `Person.Person`, and `HumanResources.EmployeeDepartmentHistory`.
* Filter for currently active department history (`EndDate` is NULL).
* Use `RANK()` to order employees by `HireDate`.
* Partition by `DepartmentID` to find the most senior members of each team.
* Return Name, Department, HireDate, and SeniorityRank.

In [ ]:
SELECT 
    p.FirstName,
    p.LastName,
    edh.DepartmentID,
    e.HireDate,
    RANK() OVER (
        PARTITION BY edh.DepartmentID 
        ORDER BY e.HireDate
    ) AS SeniorityRank
FROM HumanResources.Employee AS e
JOIN Person.Person AS p
    ON p.BusinessEntityID = e.BusinessEntityID
JOIN HumanResources.EmployeeDepartmentHistory AS edh
    ON edh.BusinessEntityID = e.BusinessEntityID
WHERE edh.EndDate IS NULL
ORDER BY edh.DepartmentID

### 7. Product Inventory Ranking by Location
**Functional Specification:**
* Select from `Production.ProductInventory` and join to `Production.Product`.
* Use `ROW_NUMBER()` to index inventory items.
* Partition by `LocationID` to handle each warehouse area separately.
* Order by `Quantity` descending to find the most stocked items.
* Return Product Name, LocationID, Quantity, and Rank.

In [ ]:
SELECT
    p.Name,
    pi2.LocationID,
    pi2.Quantity,
    ROW_NUMBER() OVER (
        PARTITION BY pi2.LocationID
        ORDER BY pi2.Quantity DESC
    ) AS Rank
FROM Production.ProductInventory AS pi2
JOIN Production.Product AS p
    ON p.ProductID = pi2.ProductID
ORDER BY pi2.LocationID

### 8. Percentile Ranking of Orders by Freight Cost
**Functional Specification:**
* Select `SalesOrderID` and `Freight` from `Sales.SalesOrderHeader`.
* Use `NTILE(100)` to calculate a percentile (1-100) for each order based on freight cost.
* Order by `Freight` ASC.
* This allows analysis of which orders fall into the top 1% of shipping costs.

In [ ]:
SELECT
    soh.SalesOrderID,
    soh.Freight,
    NTILE(100) OVER (ORDER BY soh.Freight) AS Percentile
FROM Sales.SalesOrderHeader AS soh

### 9. Dense Rank of Sales Territories by Revenue History
**Functional Specification:**
* Use `Sales.SalesTerritoryHistory` to get historical revenue (use current `SalesYTD` in `SalesTerritory`).
* Select Name and `SalesYTD` from `Sales.SalesTerritory`.
* Use `DENSE_RANK()` over the whole result set.
* Order by `SalesYTD` descending.
* Use `DENSE_RANK` to handle ties in revenue appropriately.

In [ ]:
SELECT
    st.Name,
    st.SalesYTD,
    DENSE_RANK() OVER (
        ORDER BY st.SalesYTD DESC
    ) AS Rank
FROM Sales.SalesTerritory AS st
ORDER BY st.SalesYTD DESC

### 10. Top 3 Orders per Year
**Functional Specification:**
* Extract the year from `OrderDate` in `Sales.SalesOrderHeader`.
* Use `ROW_NUMBER()` to rank orders.
* Partition by the extracted Year.
* Order by `TotalDue` descending.
* Wrap in a CTE and filter where Rank <= 3 to get the podium finishers for each year.

In [ ]:
WITH TotalDueByYears AS (
    SELECT
        YEAR(soh.OrderDate) AS OrderYear,
        soh.TotalDue,
        ROW_NUMBER() OVER (
            PARTITION BY YEAR(soh.OrderDate)
            ORDER BY soh.TotalDue DESC
        ) AS RowNumber
    FROM Sales.SalesOrderHeader AS soh
)
SELECT
    OrderYear,
    TotalDue,
    RowNumber
FROM TotalDueByYears
WHERE RowNumber <= 3